import

In [1]:
import os

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel
from typing import Dict

xionic-ko-llama

In [18]:
from openai import OpenAI

client = OpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d"
)

In [3]:
response = client.chat.completions.create(
    model="xionic-ko-llama-3-70b",
    messages=[
        {"role": "system", "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer in korean."},
        {"role": "user", "content": "아이브 리더가 활동했던 이전 걸그룹의 명칭은?"}
    ]
)

print (response)

ChatCompletion(id='cmpl-b68b656c7fa0488f933e78d9c9bf89b9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="아이브 리더인 안유진은 데뷔 전 걸그룹 IZ*ONE에서 활동했습니다. IZ*ONE은 2018년 Mnet에서 방영된 리얼리티 프로그램 '프로듀스 48'을 통해 결성된 프로젝트 걸그룹으로, 2018년 10월 29일 데뷔 했습니다. 이 그룹은 프로듀스 48에 참가한 96명의 연습생 중 최종 12명을 선발하여 구성됐으며, 안유진은 이 중 한 명으로 선발됐습니다. IZ*ONE은 2021년 4월 29일까지 활동했으며, 이후 안유진은 아이브의 리더로 데뷔하게 됩니다.", role='assistant', function_call=None, tool_calls=None), stop_reason=None)], created=1721259565, model='xionic-ko-llama-3-70b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=166, prompt_tokens=56, total_tokens=222))


문서 가져오기

In [2]:
PATH = "C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test"
Product_data = os.path.join(
    PATH, "labor_low.pdf"
)

In [3]:
# def read_prompt_template(file_path: str) -> str:
#    with open(file_path, "r") as f:
#            prompt_template = f.read()

#    return prompt_template

In [45]:
# import pandas as pd

# def read_prompt_template(file_path: str) -> str:
#     # 엑셀 파일의 모든 시트 읽기
#     xls = pd.ExcelFile(file_path)
    
#     # 모든 시트의 내용을 저장할 리스트
#     all_sheets_content = []

#     # 각 시트를 순회하며 내용 읽기
#     for sheet_name in xls.sheet_names:
#         df = pd.read_excel(file_path, sheet_name=sheet_name)
        
#         # DataFrame을 문자열로 변환하여 리스트에 추가
#         sheet_content = df.to_string(index=False)
#         all_sheets_content.append(f"Sheet: {sheet_name}\n{sheet_content}")

#     # 모든 시트의 내용을 하나의 문자열로 결합
#     prompt_template = "\n\n".join(all_sheets_content)
    
#     return prompt_template

# # 사용 예시
# file_path = Product_data
# prompt_template = read_prompt_template(file_path)
# print(prompt_template)

Sheet: 건수
 * 신제품 목록 건수 Unnamed: 1 Unnamed: 2            Unnamed: 3            Unnamed: 4 Unnamed: 5   Unnamed: 6  Unnamed: 7                                                                                                                            Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13  Unnamed: 14  Unnamed: 15  Unnamed: 16 Unnamed: 17 Unnamed: 18
         NaN        NaN        NaN                   NaN                   NaN        NaN          NaN         NaN                                                                                                                                   NaN         NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN         NaN         NaN
         NaN 구분 (건, 품목)        NaN                  2010                  2011       2012         2013      2014.0                                                                                                                            

In [6]:
# def create_chain(template_path):
#     prompt_template = read_prompt_template(template_path)
    
#     chain = LLMChain(
#         llm="xionic-ko-llama-3-70b",
#         prompt=ChatPromptTemplate.from_template(template=prompt_template),
#         max_tokens=150,
#         temperature=0.7,
#         stop=None
#     )
    
#     return chain

In [4]:
def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm = llm,
        prompt = ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path)
        ),
        output_key=output_key,
        verbose=True,        
    )

In [5]:
llm = ChatOpenAI(base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1)

c:\Users\SUNJIN\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
product_data = create_chain(
    llm = llm,
    template_path=Product_data,
    output_key="text"
)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte

In [38]:
destinations = [
    "product_data: 회사의 제품에 대해 말해줘"
]

In [39]:
destinations = "\n".join(destinations)
router_prompt_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations)
router_prompt = PromptTemplate.from_template(
    template=router_prompt_template, output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt, verbose=True)

In [40]:
multi_prompt_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains={
        "product_data": product_data
    },
    default_chain=ConversationChain(llm=router_chain, output_key="text"),
)

In [41]:
class UserRequest(BaseModel):
    user_message: str


def gernerate_answer(req: UserRequest) -> Dict[str, str]:
    context = req.model_dump()
    context["input"] = context["user_message"]
    answer = multi_prompt_chain.run(context)

    return {"answer": answer}

In [42]:
user_data = {
    "user_message": "2010년에 회사 제품이 몇개 출시됐어?"
}

In [43]:
request_instance = UserRequest(**user_data)

In [44]:
gernerate_answer(request_instance)



> Entering new LLMRouterChain chain...

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Sheet: 건수
 * 신제품 목록 건수 Unnamed: 1 Unnamed: 2            Unnamed: 3            Unnamed: 4 Unnamed: 5   Unnamed: 6  Unnamed: 7                                                                                                                            Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13  Unnamed: 14  Unnamed: 15  Unnamed: 16 Unnamed: 17 Unnamed: 18
         NaN        NaN        NaN                   NaN                   NaN        NaN          NaN         NaN                                                                                                                                   NaN         NaN          NaN          NaN          NaN          NaN          NaN          NaN          NaN         NaN         NaN
         NaN 구분 (건, 품목)        NaN                  2010                  2011       2012         2013      201

BadRequestError: Error code: 400 - {'object': 'error', 'message': 'max_tokens must be at least 1, got -32380.', 'type': 'BadRequestError', 'param': None, 'code': 400}